# 1-scraping

In [17]:
# Manejo de datos 
import pandas as pd # manejo de datos y dataframes
import numpy as np # manejo de arrays y operaciones matematicas 


# Librerias para realizar web scraping con selenium
from selenium import webdriver # webdriver permite manejar un navegador 
from webdriver_manager.chrome import ChromeDriverManager # permite instalar y mantener actualizado el driver de chrome
from selenium.webdriver.common.keys import Keys # permite simular teclas del teclado 
from selenium.webdriver.chrome.options import Options # permite configurar el driver de chrome como modo incognito o maximizar la ventana
from time import sleep # esperas entre ejecuciones de codigo


import warnings # permite ignorar los warnings de python 
warnings.filterwarnings('ignore')

In [18]:
import requests
import re
import pickle

from bs4 import BeautifulSoup

In [19]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

# para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') # empezar maximizado
opciones.add_argument('user.data-dir=selenium') # guarda las cookies
opciones.add_argument('--incognito') # incognito window

# selenium

In [ ]:
urls = []
flag = 0
flag2 = 0

driver = webdriver.Chrome()

driver.get("https://www.dia.es/") 
sleep(3)

# aceptamos las cookies 
driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
sleep(2)

# clicka productos
driver.find_element("css selector", '#app > div > div > div > div.home-view__header > div.dia-header > div.dia-header__section.dia-header__section--start > div > div > button').click()
sleep(2)

# para cada una de las categorías principales
# poner un max excesivo
for category in range(1,31):

    while 1:

        # intenta clickar
        try:
            
            driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[{category}]/a').click()
            flag2 = 0

            for subcategory in range (1,31):

                while 1:

                    try:
                        
                        urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[{category}]/ul/div[{subcategory}]/a').get_attribute("href"))
                        sleep(0.1)
                        break

                    except:

                        flag2 = 1
                        break
                    
                if flag2:
                    break
            
            sleep(1)
            break
        
        # break si se acaban
        except:

            flag = 1
            break

    if flag:

        break

driver.quit()

In [ ]:
df = pd.DataFrame(urls, columns=['url'])

df.to_csv("../data/urls-dia.csv", index=False, sep= ",")

# sopa bonita

In [ ]:
# ['url', 'supermarket', 'category', 'name', 'description', 'price', 'reference_price', 'reference_unit', 'insert_date', 'product_id']

resultados_dia = {'url': '',
                "supermarket": 'dia-es',
                "category":[],
                "name":[], 
                "description": [], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": [],
                "product_id": []}

# solo saca 10/20

In [ ]:
url = 'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001'
res = requests.get(url)
sleep(1)

print(res.status_code)

sopa = BeautifulSoup(res.content, 'html.parser')

#productos = sopa.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
productos = sopa.find_all("p", {"class": "search-product-card__product-name"})

print(f'Cantidad de productos extraídos: {len(productos)}')

print(productos[0])

print(productos[0].text)

productos


# scroll por pixel saca 20/20

In [ ]:
# Create your driver
driver = webdriver.Chrome()

# Get a page
driver.get("https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001") 
sleep(2)

Y = 1200
driver.execute_script(f"window.scrollTo(0, {Y})") 
sleep(2)

# Feed the source to BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

productos = soup.find_all("p", {"class": "search-product-card__product-name"})
precio = soup.find_all("p", {"class": "search-product-card__active-price"})
kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})

driver.quit()

print(f'Productos extraídos: {len(productos)} de 20')

print(productos[-1].text)
print(precio[-1].text)
print(kilo[-1].text)

# crea una lista de listas, cada lista es una página de productos

In [ ]:
productos = []
precio = []
kilo = []

# Create your driver
driver = webdriver.Chrome()

# Get a page
driver.get("https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001")
sleep(2)

Y = 1200
driver.execute_script(f"window.scrollTo(0, {Y})") 
sleep(2)

# Feed the source to BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

productos = soup.find_all("p", {"class": "search-product-card__product-name"})
precio = soup.find_all("p", {"class": "search-product-card__active-price"})
kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})

driver.quit()

print(f'Productos extraídos: {len(productos)} de 20')

print(productos)
print(precio)
print(kilo)

In [ ]:
for p in productos:
    print(p.text)


In [ ]:
print(len(productos))

# bucle para meter todas las páginas de un producto

In [ ]:
productos = []
precio = []
kilo = []
flag3 = 0

# Create your driver
driver = webdriver.Chrome()

param = 'c'
url = f'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/{param}/L2001'

# Get a page
driver.get(url)
sleep(2)

Y = 1200
driver.execute_script(f"window.scrollTo(0, {Y})") 
sleep(2)

# Feed the source to BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

productos.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

for i in range(2,10):

    param = f'pag-{i}'
    url = f'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/{param}/c/L2001'

    driver.get(url)
    sleep(2)

    Y = 1200
    driver.execute_script(f"window.scrollTo(0, {Y})") 
    sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    if soup.find_all("div", {"class": "plp-error-page__card"}):

        break

    productos.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
    precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
    kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

driver.quit()

print(len(productos))
print(precio)
print(kilo)

# bucle para meter varias urls

In [ ]:
productos = []
precio = []
kilo = []
flag3 = 0

# creación del driver
driver = webdriver.Chrome()

# c = página 1
param = 'c'
url = [f'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/{param}/L2001', 
       f'https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/{param}/L2206']

# get
driver.get(u)
sleep(2)

# scroll down
Y = 1200
driver.execute_script(f"window.scrollTo(0, {Y})") 
sleep(2)

# parseo html para sopa
soup = BeautifulSoup(driver.page_source, 'html.parser')

# apendea los resultados de la página 1
productos.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

# itero i para modificar url
for i in range(2,10):
    
    param = f'pag-{i}/c'

    # para cada url de la lista:
    for u in url:

        driver.get(u)
        sleep(2)

        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        if soup.find_all("div", {"class": "plp-error-page__card"}):

            break

        productos.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
        precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
        kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

driver.quit()

print(len(productos))
print(precio)
print(kilo)

In [ ]:
productos = []
precio = []
kilo = []

# creación del driver
driver = webdriver.Chrome()

for i in range(2,11):

    if i == 1:

        param = 'c'

    else:

        param = f'pag-{i}/c'

    url = [f'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/{param}/L2001', f'https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/{param}/L2206']

    for u in url:

        # get
        driver.get(u)
        sleep(2)

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parseo html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # apendea los resultados de la página 1
        productos.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
        precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
        kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

In [23]:
categoria = []
subcategoria = []
producto = []
precio = []
kilo = []
id = []

# creación del driver
driver = webdriver.Chrome()

url = ['https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001', 'https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/c/L2206']

for u in url:

        # get
        driver.get(u)
        sleep(5)

        # aceptamos las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parseo html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # apendea los resultados de la página 1
        id.append(soup.find_all("object_id"))
        categoria.append(soup.find("span", {"class": "plp-breadcrumb__first-level-category"}))
        subcategoria.append(soup.find("span", {"class": "plp-breadcrumb__second-level-category"}))
        producto.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
        precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
        kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        if botones:

                for bottom in range(2, int(botones[-1].text)+1):
                        
                        driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                        sleep(2)

                        # scroll down
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(2)

                        # parseo html para sopa
                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        # apendea los resultados de la página 1
                        id.append(soup.find_all("object_id"))
                        categoria.append(soup.find("span", {"class": "plp-breadcrumb__first-level-category"}))
                        subcategoria.append(soup.find("span", {"class": "plp-breadcrumb__second-level-category"}))
                        producto.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
                        precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
                        kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

        botones = []

driver.quit()

print(len(producto))
print(categoria)
print(subcategoria)
print(producto)
print(precio)
print(kilo)


4
[<span class="plp-breadcrumb__first-level-category">Charcutería y quesos</span>, <span class="plp-breadcrumb__first-level-category">Charcutería y quesos</span>, <span class="plp-breadcrumb__first-level-category">Charcutería y quesos</span>, <span class="plp-breadcrumb__first-level-category">Charcutería y quesos</span>]
[<span class="plp-breadcrumb__second-level-category">Jamón cocido, lacón, fiambres y mortadela</span>, <span class="plp-breadcrumb__second-level-category">Jamón cocido, lacón, fiambres y mortadela</span>, <span class="plp-breadcrumb__second-level-category">Jamón cocido, lacón, fiambres y mortadela</span>, <span class="plp-breadcrumb__second-level-category">Salchichas y elaborados</span>]
[[<p class="search-product-card__product-name">Pechuga de pavo NUESTRA ALACENA pack 2 unidades SOBRE 400 GR</p>, <p class="search-product-card__product-name">Jamón cocido extra NUESTRA ALACENA pack 2 unidades SOBRE 450 GR</p>, <p class="search-product-card__product-name">Bacon en tiras

In [ ]:
resultados_dia